# Finetuning Llama-3.2-1B for finance tasks
Basic deliverable finetuning script based on [ShawhinT finetuning example](https://github.com/ShawhinT/YouTube-Blog/tree/main/LLMs/fine-tuning)

In [ ]:
import torch
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from datasets import load_dataset
from transformers import BatchEncoding, pipeline, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers.models.llama.modeling_llama import LlamaForCausalLM
from pathlib import Path
from typing import cast


model_id = "meta-llama/Llama-3.2-1B"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer = pipe.tokenizer
base_model = cast(LlamaForCausalLM, pipe.model)

In [ ]:
list(base_model.modules())

[LlamaForCausalLM(
   (model): LlamaModel(
     (embed_tokens): Embedding(128256, 2048)
     (layers): ModuleList(
       (0-15): 16 x LlamaDecoderLayer(
         (self_attn): LlamaSdpaAttention(
           (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
           (k_proj): Linear(in_features=2048, out_features=512, bias=False)
           (v_proj): Linear(in_features=2048, out_features=512, bias=False)
           (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
           (rotary_emb): LlamaRotaryEmbedding()
         )
         (mlp): LlamaMLP(
           (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
           (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
           (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
           (act_fn): SiLU()
         )
         (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
         (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
     

In [ ]:
peft_config = LoraConfig(task_type="SEQ_CLS",
                         r=4,
                         lora_alpha=32,
                         lora_dropout=0.01,
                         target_modules = ['q_lin'])

In [ ]:
peft_model = get_peft_model(base_model, peft_config)
peft_model.print_trainable_parameters()

In [ ]:
# hyperparameters
lr = 1e-3
batch_size = 4
num_epochs = 10

In [ ]:
# define training arguments
out_dir = Path(r"D:\models\basic-Llama-3_2-LoRA")
training_args = TrainingArguments(
    output_dir=str(out_dir),
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

In [ ]:
# creater trainer object
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
    compute_metrics=compute_metrics,
)

# train model
trainer.train()